In [1]:
from graph import Graph, Node
from a_star import AStar
import time
start = time.time()

In [ ]:
from aas import model
import json
import aas.adapter.json
import aas.adapter.xml
from aas import model
from aas.model import Asset, AssetAdministrationShell, Submodel
import pandas as pd
import os

In [3]:
from pymongo import MongoClient

# client = MongoClient('localhost', 27017)
client = MongoClient(host='localhost', port= 27017)

# db = client.test_database
db = client['test_db'] # test-db라는 이름의 데이터베이스에 접속
collection_currency = db['collection_currency']

## 라이브러리를 이용하여 위도,경도로 직선 거리 구하기

In [4]:
from geopy.geocoders import Nominatim

def geocoding(address):
    geolocoder = Nominatim(user_agent = 'America', timeout=None)
    geo = geolocoder.geocode(address)
    crd = {"lat": float(geo.latitude), "lng": float(geo.longitude)}

    return crd


In [ ]:
Forging_loc = pd.read_csv('D:/AAS_kse/aas/Standard_data/공작기계_단조.csv', encoding= 'cp949')
Forging_loc = list(Forging_loc['Location'].unique())    
Forging_loc

['6666 W 66th St, Chicago, Illinois ']

In [6]:
for_cor = geocoding(Forging_loc)

In [7]:
for_cor

{'lat': 41.7745409, 'lng': -87.6299037}

In [8]:
import numbers
import math

class GeoUtil:
    """
    Geographical Utils
    """
    @staticmethod
    def degree2radius(degree):
        return degree * (math.pi/180)
    
    def get_harversion_distance(x1, y1, x2, y2, round_decimal_digits=5):
        """
        경위도 (x1,y1)과 (x2,y2) 점의 거리를 반환
        Harversion Formula 이용하여 2개의 경위도간 거래를 구함(단위:Km)
        """
        if x1 is None or y1 is None or x2 is None or y2 is None:
            return None
        assert isinstance(x1, numbers.Number) and -180 <= x1 and x1 <= 180
        assert isinstance(y1, numbers.Number) and  -90 <= y1 and y1 <=  90
        assert isinstance(x2, numbers.Number) and -180 <= x2 and x2 <= 180
        assert isinstance(y2, numbers.Number) and  -90 <= y2 and y2 <=  90

        R = 6371 # 지구의 반경(단위: km)
        dLon = GeoUtil.degree2radius(x2-x1)    
        dLat = GeoUtil.degree2radius(y2-y1)

        a = math.sin(dLat/2) * math.sin(dLat/2) \
            + (math.cos(GeoUtil.degree2radius(y1)) \
              *math.cos(GeoUtil.degree2radius(y2)) \
              *math.sin(dLon/2) * math.sin(dLon/2))
        b = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        return round(R * b, round_decimal_digits)



In [9]:
GeoUtil.get_harversion_distance(-96.60464, 32.24846, -79.66870, 40.53306)

1769.25238

## Manufacturing Process 배치 

In [10]:
## 단조 프레스 배치
for r in collection_currency.find({"$and":[{"assetAdministrationShells.idShort":"ForgingPress"} 
                                           ]}):
    print(r)
Forging_AAS_Seosan = r

{'_id': ObjectId('659e04f6926451a6f1528aa4'), 'assetAdministrationShells': [{'idShort': 'ForgingPress', 'modelType': {'name': 'AssetAdministrationShell'}, 'identification': {'id': 'http://localhost:8080/aashell', 'idType': 'Custom'}, 'asset': {'keys': [{'type': 'Asset', 'idType': 'Custom', 'value': 'localhost:8080/asset/', 'local': True}]}, 'submodels': [{'keys': [{'type': 'Submodel', 'idType': 'Custom', 'value': 'localhost:8080/Submodel/OperationalData', 'local': True}]}, {'keys': [{'type': 'Submodel', 'idType': 'Custom', 'value': 'localhost:8080/Submodel/Documentation', 'local': True}]}, {'keys': [{'type': 'Submodel', 'idType': 'Custom', 'value': 'localhost:8080/Submodel/Category', 'local': True}]}, {'keys': [{'type': 'Submodel', 'idType': 'Custom', 'value': 'localhost:8080/Submodel/Size', 'local': True}]}, {'keys': [{'type': 'Submodel', 'idType': 'Custom', 'value': 'localhost:8080/Submodel/Nameplate', 'local': True}]}, {'keys': [{'type': 'Submodel', 'idType': 'Custom', 'value': 'loc

In [11]:
data = pd.read_csv('D:/AAS_kse/aas/Standard_data/공작기계_그라인더.csv', encoding= 'cp949')

data

,Type,Equipment,name,Brand,Table_Size,Grinding wheel power,Grinding_Speed,Location,Company
0,Flat surface grinder,GRINDER,20AD-36,Blanchard,None,37KW,720RPM,"240 E Rosecrans Ave, Gardena, California",TCI Precision Metals
1,Flat surface grinder,GRINDER,22AD-42,Blanchard,None,37KW,720RPM,"240 E Rosecrans Ave, Gardena, California",TCI Precision Metals
2,Flat surface grinder,GRINDER,32HD-60,SPNK,None,74KW,720RPM,"240 E Rosecrans Ave, Gardena, California",TCI Precision Metals
3,Flat surface grinder,GRINDER,BD-12,SKNK,1219x609mm,11.9KW,None,"240 E Rosecrans Ave, Gardena, California",TCI Precision Metals
4,Flat surface grinder,GRINDER,ID-400,OVERBECK,425x475mm,None,None,"196 Alwine Rd, Saxonburg, Pennsylvania",PENN UNITED
5,Cylindrical grinder,GRINDER,75-s2,USACH,2091x1481mm,None,None,"196 Alwine Rd, Saxonburg, Pennsylvania",PENN UNITED
6,Cylindrical grinder,GRINDER,LIBERTY II,Parker Majestic,127x965.2mm,18.5KW,1000RPM,"196 Alwine Rd, Saxonburg, Pennsylvania",PENN UNITED
7,Cylindrical grinder,GRINDER,CN148P4,Rollomatic,1830x2040mm,1.5KW,3000RPM,"196 Alwine Rd, Saxonburg, Pennsylvania",PENN UNITED
8,Flat surface grinder,GRINDER,s35-400,Hauser,600x380mm,7.5KW,4500RPM,"196 Alwine Rd, Saxonburg, Pennsylvania",PENN UNITED
9,Flat surface grinder,GRINDER,FSG-3A1224H,CHEVALIER,305 x 610mm,3.7KW,1100RPM,450 Whitney Road West,Nifty Bar


In [12]:
Grinder_Location = list(data['Location'].unique())
Grinder_Location

['240 E Rosecrans Ave, Gardena, California',
 '196 Alwine Rd, Saxonburg, Pennsylvania',
 '450 Whitney Road West',
 '2904 Scott Blvd, Santa Clara, California',
 '931 Merwin Road, Pennsylvania',
 '2019 Wood-Bridge Blvd, Bowling Green, Ohio']

In [20]:
#Material address
#1 : 5105 Newport Drive Rolling Meadows, Illinois

#Crankshaft manufacturing company
#2828 Fallon Dr,Sherman,Texas


In [14]:
# crankshaft를 제작하는 안성과 기계 사이들 간의 거리 구하기

def location_distance(sta_loaction , arr_location):

    Start_Location = geocoding(sta_loaction)
    Arrive_Location = geocoding(arr_location)

    return(round(GeoUtil.get_harversion_distance(Start_Location['lng'], Start_Location['lat'], Arrive_Location['lng'], Arrive_Location['lat'])))

In [15]:
for i in range(len(Grinder_Location)):
    print("{}->{} : ".format(Grinder_Location[i],crankshaft_man) ,location_distance(Grinder_Location[i],crankshaft_man))

240 E Rosecrans Ave, Gardena, California->2828 Fallon Dr,Sherman,Texas :  2006
196 Alwine Rd, Saxonburg, Pennsylvania->2828 Fallon Dr,Sherman,Texas :  1676
450 Whitney Road West->2828 Fallon Dr,Sherman,Texas :  1878
2904 Scott Blvd, Santa Clara, California->2828 Fallon Dr,Sherman,Texas :  2326
931 Merwin Road, Pennsylvania->2828 Fallon Dr,Sherman,Texas :  1678
2019 Wood-Bridge Blvd, Bowling Green, Ohio->2828 Fallon Dr,Sherman,Texas :  1425


In [36]:
for i in range(len(Grinder_Location)):
    print("{}->{} : ".format('11755 S Austin Ave, Alsip, Illinois',Grinder_Location[i]) ,location_distance('11755 S Austin Ave, Alsip, Illinois',Grinder_Location[i]))

11755 S Austin Ave, Alsip, Illinois->240 E Rosecrans Ave, Gardena, California :  2803
11755 S Austin Ave, Alsip, Illinois->196 Alwine Rd, Saxonburg, Pennsylvania :  674
11755 S Austin Ave, Alsip, Illinois->450 Whitney Road West :  822
11755 S Austin Ave, Alsip, Illinois->2904 Scott Blvd, Santa Clara, California :  2952
11755 S Austin Ave, Alsip, Illinois->931 Merwin Road, Pennsylvania :  690
11755 S Austin Ave, Alsip, Illinois->2019 Wood-Bridge Blvd, Bowling Green, Ohio :  348


In [23]:
GeoUtil.get_harversion_distance(-96.60464, 32.24846, -79.66870, 40.53306)

1769.25238

In [24]:
#CrankShaft_Arrive
arr_loc = "2828 Fallon Dr,Sherman,Texas"

In [25]:
###그라인더 공정 최단경로 알고리즘 

def run():
    # Create graph
    graph = Graph()
    # Add vertices
    graph.add_node(Node("CrankShaft_Texas_Start", (1,1))) # S
    graph.add_node(Node("Grinder_AAS_California_Gardena",(2,1))) # B
    graph.add_node(Node("Grinder_AAS_Pennsylvania_Saxonburg", (2,2))) # C
    graph.add_node(Node("Grinder_AAS_NewYork_Penfield", (2,3))) # D 
    graph.add_node(Node("Grinder_AAS_California_Santa Clara",(2,4)))
    graph.add_node(Node("Grinder_AAS_Pennsylvania_New Kensington",(2,5)))
    graph.add_node(Node("Grinder_AAS_Ohio_Bowling Green",(2,6)))
    graph.add_node(Node("CrankShaft_Texas_Arrive", (3,1)))

    # Add edges
    graph.add_edge("CrankShaft_Texas_Start", "Grinder_AAS_California_Gardena", location_distance(Grinder_Location[0],arr_loc)) #S B
    graph.add_edge("CrankShaft_Texas_Start", "Grinder_AAS_Pennsylvania_Saxonburg", location_distance(Grinder_Location[1],arr_loc)) #S D
    graph.add_edge("CrankShaft_Texas_Start", "Grinder_AAS_NewYork_Penfield", location_distance(Grinder_Location[2],arr_loc))# B E
    graph.add_edge("CrankShaft_Texas_Start", "Grinder_AAS_California_Santa Clara", location_distance(Grinder_Location[3],arr_loc))# B E
    graph.add_edge("CrankShaft_Texas_Start", "Grinder_AAS_Pennsylvania_New Kensington", location_distance(Grinder_Location[4],arr_loc))# B E
    graph.add_edge("CrankShaft_Texas_Start", "Grinder_AAS_Ohio_Bowling Green", location_distance(Grinder_Location[5],arr_loc))# B E
    
    graph.add_edge("Grinder_AAS_California_Gardena", "CrankShaft_Texas_Arrive", location_distance(Grinder_Location[0],arr_loc))# B E
    graph.add_edge("Grinder_AAS_Pennsylvania_Saxonburg", "CrankShaft_Texas_Arrive", location_distance(Grinder_Location[1],arr_loc))# B E
    graph.add_edge("Grinder_AAS_NewYork_Penfield", "CrankShaft_Texas_Arrive", location_distance(Grinder_Location[2],arr_loc))# B E
    graph.add_edge("Grinder_AAS_California_Santa Clara", "CrankShaft_Texas_Arrive", location_distance(Grinder_Location[3],arr_loc))# B E
    graph.add_edge("Grinder_AAS_Pennsylvania_New Kensington", "CrankShaft_Texas_Arrive", location_distance(Grinder_Location[4],arr_loc))# B E
    graph.add_edge("Grinder_AAS_Ohio_Bowling Green", "CrankShaft_Texas_Arrive", location_distance(Grinder_Location[5],arr_loc))# B E
    


    # Execute the algorithm
    alg = AStar(graph, "CrankShaft_Texas_Start","CrankShaft_Texas_Arrive")
    path, path_length = alg.search()
    print(" -> ".join(path))

#if __name__ == '__main__':
run()

CrankShaft_Texas_Start -> Grinder_AAS_Ohio_Bowling Green -> CrankShaft_Texas_Arrive


In [27]:
lathe_data = pd.read_csv('D:/AAS_kse/aas/Standard_data/공작기계_선반.csv', encoding= 'cp949')

lathe_data

,Type,Equipment,name,Brand,Size,Swing over bed,DC,Spindle_Speed,Spindle_Power,Location,Company
0,CNC LATHE,Machine Tool,SL-2H,Mori Seiki,3988W x 1651L x 1880H,Ø569,406mm,4000RPM,15KW,"323 E Roosevelt Ave, Zeeland, Michigan",PROCAM
1,CNC LATHE,Machine Tool,HL-4,HAAS,559W x 2337L x 1143H,Ø610,864mm,3500RPM,22KW,"323 E Roosevelt Ave, Zeeland, Michigan",PROCAM
2,CNC LATHE,Machine Tool,SL 25(500),Samsung,None,Ø521,378mm,3500RPM,22KW,"5349 W 161st St, Cleveland, Ohio",Criterion
3,CNC LATHE,Machine Tool,Puma 2600 SY II,Doosan,1945W x 3560L x 2163H,Ø376,760mm,4000RPM,22KW,"5349 W 161st St, Cleveland, Ohio",Criterion
4,LATHE,Machine Tool,T-2,HWACHEON,2180W x 3100L x 2100H,Ø880,460mm,5000RPM,15KW,"1043 Kaiser Rd SW, Olympia, Washington",dti EXACT
5,CNC LATHE,Machine Tool,LZ-01R,Miyano,2896W x 1448L x 1702H,Ø320,89mm,6000RPM,10KW,"7081 International Dr, Louisville, Kentucky",MIYAMA
6,CNC LATHE,Machine Tool,STORM A50,Clausing,1016W x 610L x 2403H,Ø406,171mm,4000RPM,4KW,"7081 International Dr, Louisville, Kentucky",MIYAMA
7,CNC LATHE,Machine Tool,GTC-20,Leadwell,None,Ø380,300mm,5000RPM,11KW,"7081 International Dr, Louisville, Kentucky",MIYAMA
8,CNC LATHE,Machine Tool,Lynx 220C,Doosan,1600W x .2325L x 1655H,Ø320,305mm,4000RPM,15KW,"196 Alwine Rd, Saxonburg, Pennsylvania",PENN UNITED
9,CNC LATHE,Machine Tool,Puma 300C,Doosan,4572W x 2159L x 2311H,Ø570,630mm,2800RPM,26KW,"196 Alwine Rd, Saxonburg, Pennsylvania",PENN UNITED


In [28]:
lathe_location = list(lathe_data['Location'].unique())

lathe_location

['323 E Roosevelt Ave, Zeeland, Michigan',
 '5349 W 161st St, Cleveland, Ohio',
 '1043 Kaiser Rd SW, Olympia, Washington',
 '7081 International Dr, Louisville, Kentucky',
 '196 Alwine Rd, Saxonburg, Pennsylvania',
 '11663 McKinney Rd, Titusville, Pennsylvania',
 '931 Merwin Road, Pennsylvania',
 '10908 County Rd 419, Texas',
 '11755 S Austin Ave, Alsip, Illinois',
 '6811 E Mission Ave, Spokane Valley, Washington']

In [30]:
for i in range(len(lathe_location)):
    print("{}->{} : ".format('6666 W 66th St, Chicago, Illinois',lathe_location[i]) ,location_distance('6666 W 66th St, Chicago, Illinois',lathe_location[i]))

6666 W 66th St, Chicago, Illinois->323 E Roosevelt Ave, Zeeland, Michigan :  183
6666 W 66th St, Chicago, Illinois->5349 W 161st St, Cleveland, Ohio :  493
6666 W 66th St, Chicago, Illinois->1043 Kaiser Rd SW, Olympia, Washington :  2834
6666 W 66th St, Chicago, Illinois->7081 International Dr, Louisville, Kentucky :  431
6666 W 66th St, Chicago, Illinois->196 Alwine Rd, Saxonburg, Pennsylvania :  672
6666 W 66th St, Chicago, Illinois->11663 McKinney Rd, Titusville, Pennsylvania :  667
6666 W 66th St, Chicago, Illinois->931 Merwin Road, Pennsylvania :  688
6666 W 66th St, Chicago, Illinois->10908 County Rd 419, Texas :  1129
6666 W 66th St, Chicago, Illinois->11755 S Austin Ave, Alsip, Illinois :  11
6666 W 66th St, Chicago, Illinois->6811 E Mission Ave, Spokane Valley, Washington :  2411


In [41]:
location_distance('6666 W 66th St, Chicago, Illinois',lathe_location[3])

430

In [29]:
###CNC Lathe  최단경로 알고리즘 

def run():
    # Create graph
    graph = Graph()
    # Add vertices
    graph.add_node(Node("CrankShaft_Texas_Start", (1,1))) # S
    graph.add_node(Node("CNC_Lathe_AAS_Michigan_Zeeland",(2,1))) # B
    graph.add_node(Node("CNC_Lathe_AAS_Ohio_Cleveland", (2,2))) # C
    graph.add_node(Node("CNC_Lathe_AAS_Washington_Olympia", (2,3))) # D
    graph.add_node(Node("CNC_Lathe_AAS_Kentucky_Louisville", (2,4))) # D
    graph.add_node(Node("CNC_Lathe_AAS_Pennsylvania_Saxonburg", (2,5))) # D
    graph.add_node(Node("CNC_Lathe_AAS_Pennsylvania_Titusville", (2,6))) # D
    graph.add_node(Node("CNC_Lathe_AAS_Pennsylvania_New Kensington", (2,7))) # D
#    graph.add_node(Node("CNC_Lathe_AAS_Texas_Navasota", (2,8))) # D
    graph.add_node(Node("CrankShaft_Texas_Arrive", (3,1)))

    # Add edges
    graph.add_edge("CrankShaft_Texas_Start", "CNC_Lathe_AAS_Michigan_Zeeland", location_distance(lathe_location[0],arr_loc)) #S B
    graph.add_edge("CrankShaft_Texas_Start", "CNC_Lathe_AAS_Ohio_Cleveland",location_distance(lathe_location[1],arr_loc) ) #S D
    graph.add_edge("CrankShaft_Texas_Start", "CNC_Lathe_AAS_Washington_Olympia", location_distance(lathe_location[2],arr_loc))# B E
    graph.add_edge("CrankShaft_Texas_Start", "CNC_Lathe_AAS_Kentucky_Louisville", location_distance(lathe_location[3],arr_loc))# B E
    graph.add_edge("CrankShaft_Texas_Start", "CNC_Lathe_AAS_Pennsylvania_Saxonburg", location_distance(lathe_location[4],arr_loc))# B E
    graph.add_edge("CrankShaft_Texas_Start", "CNC_Lathe_AAS_Pennsylvania_Titusville", location_distance(lathe_location[5],arr_loc))
    graph.add_edge("CrankShaft_Texas_Start", "CNC_Lathe_AAS_Pennsylvania_New Kensington", location_distance(lathe_location[6],arr_loc))
#    graph.add_edge("CrankShaft_Texas_Start", "CNC_Lathe_AAS_Texas_Navasota", location_distance(lathe_location[7],arr_loc))
    

    graph.add_edge("CNC_Lathe_AAS_Michigan_Zeeland", "CrankShaft_Texas_Arrive", location_distance(lathe_location[0],arr_loc))# B E
    graph.add_edge("CNC_Lathe_AAS_Ohio_Cleveland", "CrankShaft_Texas_Arrive", location_distance(lathe_location[1],arr_loc))
    graph.add_edge("CNC_Lathe_AAS_Washington_Olympia", "CrankShaft_Texas_Arrive", location_distance(lathe_location[2],arr_loc))
    graph.add_edge("CNC_Lathe_AAS_Kentucky_Louisville", "CrankShaft_Texas_Arrive", location_distance(lathe_location[3],arr_loc))
    graph.add_edge("CNC_Lathe_AAS_Pennsylvania_Saxonburg", "CrankShaft_Texas_Arrive", location_distance(lathe_location[4],arr_loc))
    graph.add_edge("CNC_Lathe_AAS_Pennsylvania_Titusville", "CrankShaft_Texas_Arrive", location_distance(lathe_location[5],arr_loc))
    graph.add_edge("CNC_Lathe_AAS_Pennsylvania_New Kensington", "CrankShaft_Texas_Arrive", location_distance(lathe_location[6],arr_loc))
    #graph.add_edge("CNC_Lathe_AAS_Michigan_Zeeland", "CrankShaft_Texas_Arrive", 175)
    
#Texas CNC Lathe가 제일 가깝긴 하지만 설비의 사양이 맞지 않기 때문에 제외 시킴
    



    # Execute the algorithm
    alg = AStar(graph, "CrankShaft_Texas_Start","CrankShaft_Texas_Arrive")
    path, path_length = alg.search()
    print(" -> ".join(path))

#if __name__ == '__main__':
run()

CrankShaft_Texas_Start -> CNC_Lathe_AAS_Kentucky_Louisville -> CrankShaft_Texas_Arrive


In [31]:
milling_data = pd.read_csv('D:/AAS_kse/aas/Standard_data/공작기계_밀링.csv', encoding= 'cp949')

milling_data

,Type,Equipment,name,Brand,Size,Table_Size,Spindle_motor,Spindle_speed,Travel_distance(X/Y/Z),Location,Company
0,Vertical Machining Center,Machine Tool,VM5020,Milltronics,2845W x 2764L x 2591H,1320mm x 508mm,11KW,10000RPM,1270mm x 508mm x 508mm,"323 E Roosevelt Ave, Zeeland, Michigan",PROCAM
1,Vertical Machining Center,Machine Tool,VM16,Milltronics,2362W x 2362L x 2794H,1143mm x 508mm,18KW,8000RPM,762mm x 508mm x 508mm,"323 E Roosevelt Ave, Zeeland, Michigan",PROCAM
2,Horizontal Machining Center,Machine Tool,ES-5-4-AX,HAAS,None,1321mm x 508mm,15KW,12000RPM,1016mm x 559mm x 457mm,"323 E Roosevelt Ave, Zeeland, Michigan",PROCAM
3,Vertical Machining Center,Machine Tool,VM30,Milltronics,None,1676mm x 762 mm,18KW,8000RPM,1524mm x 762mm x 610mm,"323 E Roosevelt Ave, Zeeland, Michigan",PROCAM
4,Horizontal Machining Center,Machine Tool,HS-1,HAAS,3632W x 2794L x 2820H,None,15KW,7500RPM,610mm x 508mm x 559mm,"323 E Roosevelt Ave, Zeeland, Michigan",PROCAM
5,Vertical Machining Center,Machine Tool,Series II,Bridgeport,2432W x 1753L x 2108H,1473mm x 279mm,3KW,3500 RPM,851mm x 381mm x 406mm,"323 E Roosevelt Ave, Zeeland, Michigan",PROCAM
6,Vertical Machining Center,Machine Tool,RW-14,Milltronics,1854W x 1854H x 2235L,762mm x 356mm,11KW,8000RPM,610mm x 356mm x 521mm,"323 E Roosevelt Ave, Zeeland, Michigan",PROCAM
7,Horizontal Machining Center,Machine Tool,EC-500,HAAS,5840W x 3200L x 3000H,928mm x 749mm,22KW,7500RPM,813mm x 635mm 711mm,"323 E Roosevelt Ave, Zeeland, Michigan",PROCAM
8,Vertical Machining Center,Machine Tool,VM 40 II,HITACHI SEIKI,2330W x 2085L x 2550H,760mm x 410 mm,None,8000RPM,559mm x 406mm x 406mm,"5349 W 161st St, Cleveland, Ohio",Criterion
9,Vertical Machining Center,Machine Tool,MyCenter-2Xi,Kitamura,2165W x 24232L x 2825H,768mm x 355 mm,11KW,10000RPM,511mm x 356mm x 460mm,"5349 W 161st St, Cleveland, Ohio",Criterion


In [33]:
milling_location = list(milling_data['Location'].unique())

milling_location

['323 E Roosevelt Ave, Zeeland, Michigan',
 '5349 W 161st St, Cleveland, Ohio',
 '1043 Kaiser Rd SW, Olympia, Washington',
 '7081 International Dr, Louisville, Kentucky',
 '196 Alwine Rd, Saxonburg, Pennsylvania',
 '2019 Wood-Bridge Blvd, Bowling Green, Ohio',
 '931 Merwin Road, Pennsylvania',
 '10908 County Rd 419, Texas',
 '11755 S Austin Ave, Alsip, Illinois',
 '6811 E Mission Ave, Spokane Valley, Washington']

In [32]:
location_distance(milling_location[0],"6666 W 66th St, Chicago, Illinois ")

181

In [33]:
###Machining Tool 최단경로 알고리즘 

def run():
    # Create graph
    graph = Graph()
    # Add vertices
    graph.add_node(Node("CrankShaft_Texas_Start", (1,1))) # S
    graph.add_node(Node("MachineTool_AAS_Michigan_Zeeland",(2,1))) # B
    graph.add_node(Node("MachineTool_AAS_Ohio_Cleveland", (2,2))) # C
    graph.add_node(Node("MachineTool_AAS_Washington_Olympia", (2,3))) # D
    graph.add_node(Node("MachineTool_AAS_Kentucky_Louisville", (2,4))) # D
    graph.add_node(Node("MachineTool_AAS_Pennsylvania_Saxonburg", (2,5))) # D
    graph.add_node(Node("MachineTool_AAS_Ohio_Bowling Green", (2,6))) # D
    graph.add_node(Node("MachineTool_AAS_Pennsylvania_New Kensington", (2,7))) # D
    graph.add_node(Node("MachineTool_AAS_Texas_Navasota", (2,8))) # D
    graph.add_node(Node("CrankShaft_Texas_Arrive", (3,1)))

    # Add edges
    graph.add_edge("CrankShaft_Texas_Start", "MachineTool_AAS_Michigan_Zeeland", location_distance(milling_location[0],arr_loc)) #S B
    graph.add_edge("CrankShaft_Texas_Start", "MachineTool_AAS_Ohio_Cleveland",location_distance(milling_location[1],arr_loc) ) #S D
    graph.add_edge("CrankShaft_Texas_Start", "MachineTool_AAS_Washington_Olympia", location_distance(milling_location[2],arr_loc))# B E
    graph.add_edge("CrankShaft_Texas_Start", "MachineTool_AAS_Kentucky_Louisville", location_distance(milling_location[3],arr_loc))# B E
    graph.add_edge("CrankShaft_Texas_Start", "MachineTool_AAS_Pennsylvania_Saxonburg", location_distance(milling_location[4],arr_loc))# B E
    graph.add_edge("CrankShaft_Texas_Start", "MachineTool_AAS_Ohio_Bowling Green", location_distance(milling_location[5],arr_loc))# B E
    graph.add_edge("CrankShaft_Texas_Start", "MachineTool_AAS_Pennsylvania_New Kensington", location_distance(milling_location[6],arr_loc))
    graph.add_edge("CrankShaft_Texas_Start", "MachineTool_AAS_Texas_Navasota", location_distance(milling_location[7],arr_loc))
    

    graph.add_edge("MachineTool_AAS_Michigan_Zeeland", "CrankShaft_Texas_Arrive", location_distance(milling_location[0],arr_loc))# B E
    graph.add_edge("MachineTool_AAS_Ohio_Cleveland", "CrankShaft_Texas_Arrive", location_distance(milling_location[1],arr_loc))
    graph.add_edge("MachineTool_AAS_Washington_Olympia", "CrankShaft_Texas_Arrive", location_distance(milling_location[2],arr_loc))
    graph.add_edge("MachineTool_AAS_Kentucky_Louisville", "CrankShaft_Texas_Arrive", location_distance(milling_location[3],arr_loc))
    graph.add_edge("MachineTool_AAS_Pennsylvania_Saxonburg", "CrankShaft_Texas_Arrive", location_distance(milling_location[4],arr_loc))
    graph.add_edge("MachineTool_AAS_Ohio_Bowling Green", "CrankShaft_Texas_Arrive", location_distance(milling_location[5],arr_loc))
    graph.add_edge("MachineTool_AAS_Pennsylvania_New Kensington", "CrankShaft_Texas_Arrive", location_distance(milling_location[6],arr_loc))
    graph.add_edge("MachineTool_AAS_Texas_Navasota", "CrankShaft_Texas_Arrive", location_distance(milling_location[7],arr_loc))
    
#Texas CNC Lathe가 제일 가깝긴 하지만 설비의 사양이 맞지 않기 때문에 제외 시킴
    



    # Execute the algorithm
    alg = AStar(graph, "CrankShaft_Texas_Start","CrankShaft_Texas_Arrive")
    path, path_length = alg.search()
    print(" -> ".join(path))

#if __name__ == '__main__':
run()

CrankShaft_Texas_Start -> MachineTool_AAS_Texas_Navasota -> CrankShaft_Texas_Arrive


In [40]:
#material_location
metal_loc = "1835 Dueber Ave. SW Canton, Ohio"


#Forging_Location
forging_location = "6666 W 66th St, Chicago, Illinois"


location_distance(metal_loc,forging_location)

538

In [45]:
def run():
    # Create graph
    graph = Graph()
    # Add vertices
#    graph.add_node(Node("Materials_Ohio_SW Canton", (1,1))) 
    
    graph.add_node(Node("Forging_AAS_Illinois_Chicago", (1,1))) 
    
    graph.add_node(Node("CNC_Lathe_AAS_Michigan_Zeeland",(2,1))) # B
    graph.add_node(Node("CNC_Lathe_AAS_Ohio_Cleveland", (2,2))) # C
    graph.add_node(Node("CNC_Lathe_AAS_Washington_Olympia", (3,3))) # D
    graph.add_node(Node("CNC_Lathe_AAS_Kentucky_Louisville", (2,4))) # D
    graph.add_node(Node("CNC_Lathe_AAS_Pennsylvania_Saxonburg", (2,5))) # D
    graph.add_node(Node("CNC_Lathe_AAS_Pennsylvania_Titusville", (2,6))) # D
    graph.add_node(Node("CNC_Lathe_AAS_Pennsylvania_New Kensington", (2,7))) # D  
    
    graph.add_node(Node("Forging_AAS_Illinois_Chicago_Arrive", (3,1)))

    
    graph.add_edge("Forging_AAS_Illinois_Chicago", "CNC_Lathe_AAS_Michigan_Zeeland", location_distance(forging_location, lathe_location[0])) #S B
    graph.add_edge("Forging_AAS_Illinois_Chicago", "CNC_Lathe_AAS_Ohio_Cleveland", location_distance(forging_location, lathe_location[1])) #S D
    graph.add_edge("Forging_AAS_Illinois_Chicago", "CNC_Lathe_AAS_Washington_Olympia", location_distance(forging_location, lathe_location[2])) #S D
    graph.add_edge("Forging_AAS_Illinois_Chicago", "CNC_Lathe_AAS_Kentucky_Louisville", location_distance(forging_location, lathe_location[3])) #S D
    graph.add_edge("Forging_AAS_Illinois_Chicago", "CNC_Lathe_AAS_Pennsylvania_Saxonburg", location_distance(forging_location, lathe_location[4])) #S D
    graph.add_edge("Forging_AAS_Illinois_Chicago", "CNC_Lathe_AAS_Pennsylvania_Titusville", location_distance(forging_location, lathe_location[5])) #S D
    graph.add_edge("Forging_AAS_Illinois_Chicago", "CNC_Lathe_AAS_Pennsylvania_New Kensington", location_distance(forging_location, lathe_location[6])) #S D

    graph.add_edge("CNC_Lathe_AAS_Michigan_Zeeland","Forging_AAS_Illinois_Chicago_Arrive", location_distance(forging_location, lathe_location[0])) #S B
    graph.add_edge("CNC_Lathe_AAS_Ohio_Cleveland", "Forging_AAS_Illinois_Chicago_Arrive", location_distance(forging_location, lathe_location[1])) #S D
    graph.add_edge("CNC_Lathe_AAS_Washington_Olympia", "Forging_AAS_Illinois_Chicago_Arrive", location_distance(forging_location, lathe_location[2])) #S D
    graph.add_edge("CNC_Lathe_AAS_Kentucky_Louisville", "Forging_AAS_Illinois_Chicago_Arrive",  location_distance(forging_location, lathe_location[3])) #S D
    graph.add_edge("CNC_Lathe_AAS_Pennsylvania_Saxonburg", "Forging_AAS_Illinois_Chicago_Arrive", location_distance(forging_location, lathe_location[4])) #S D
    graph.add_edge("CNC_Lathe_AAS_Pennsylvania_Titusville", "Forging_AAS_Illinois_Chicago_Arrive",  location_distance(forging_location, lathe_location[5])) #S D
    graph.add_edge("CNC_Lathe_AAS_Pennsylvania_New Kensington", "Forging_AAS_Illinois_Chicago_Arrive", location_distance(forging_location, lathe_location[6])) #S D
    

    
    # Execute the algorithm
    alg = AStar(graph, "Forging_AAS_Illinois_Chicago", "Forging_AAS_Illinois_Chicago_Arrive")
    path, path_length = alg.search()
    print(" -> ".join(path))
    print(f"Length of the path: {path_length}")

#if __name__ == '__main__':
run()

Forging_AAS_Illinois_Chicago -> CNC_Lathe_AAS_Michigan_Zeeland -> Forging_AAS_Illinois_Chicago_Arrive
Length of the path: 362


In [7]:
def run():
    # Create graph
    graph = Graph()
    # Add vertices
    graph.add_node(Node(Forging_AAS, (1,1))) # S
    graph.add_node(Node(MachineTool_AAS_Sejong,(1,2))) # B
    graph.add_node(Node(MachineTool_AAS_Goryeong, (1,3))) # C
    graph.add_node(Node(Grinding_AAS_jinju, (2,1))) # D

    # Add edges
    graph.add_edge(Forging_AAS, MachineTool_AAS_Goryeong, 52) #S B
    graph.add_edge(Forging_AAS, MachineTool_AAS_Sejong, 83) #S D
    graph.add_edge(MachineTool_AAS_Goryeong, Grinding_AAS_jinju, 147)# B E
    graph.add_edge(MachineTool_AAS_Sejong, Grinding_AAS_jinju, 122) #C #G
  

    # Execute the algorithm
    alg = AStar(graph, Forging_AAS, Grinding_AAS_jinju)
    path, path_length = alg.search()
    print(" -> ".join(path))
    print(f"Length of the path: {path_length}")

if __name__ == '__main__':
    run()

NameError: name 'Forging_AAS' is not defined